In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats as st
pd.options.mode.chained_assignment = None   # off the warning when modifying the first mv_lag in each stock group

In [ ]:
# 读取股票收益表以及公司资产负债表
path_srm = "ret_group_dum.sas7bdat"
path_balance ="cn_balance_sheet.sas7bdat"
#print("### loading data")

srm = pd.read_sas(path_srm,
                 format='sas7bdat',
                 encoding='iso-8859-1')  
balance = pd.read_sas(path_balance,
                 format='sas7bdat',
                 encoding='iso-8859-1')  
#print("### data loaded")

In [53]:



# 起止年份
begdate = 2000
enddate = 2015
market = [1,4,16]
# 需要去除的金融行业代码
exclude_industry = ["J66","J67", "J68", "J69"]  #financila industry  "NNINDCD"

### loading data
### data loaded


In [2]:
def monthly_return(inset):   #处理原始月频收益数据，去除金融行业
    print("### processing monthly return")
    keep = ["STKCD", 'TRDMNT', 'year','yearr', 'month','status_decision',
            'NNINDCD','MSMVTTL','MCLSPRC','MARKETTYPE','date','MRETWD','MSMVOSD','mv','mv_lag']
    inset.sort_values(by = ["STKCD","TRDMNT"],ascending=True,inplace = True)
    inset.query("NNINDCD != @exclude_industry & MARKETTYPE == @market & (@enddate>=year >= @begdate)& MRETWD==MRETWD & MSMVOSD==MSMVOSD",
              inplace=True)   #using query to speed up
    inset["mv"] = inset["MSMVOSD"]
    inset['mv_lag'] = inset["MSMVOSD"].shift(1)
    inset.loc[inset.groupby('STKCD')['mv_lag'].head(1).index, 'mv_lag'] = np.nan #当期的市值参考值为上一期的市值
    inset["date"] = inset['year']*10000 +inset["month"]*100+28
    inset.loc[inset['month']<7,"yearr"] = inset["year"]-1
    inset.loc[inset['month']>=7,"yearr"] = inset["year"]
    inset = inset[keep]
    inset.reset_index(drop=True,inplace=True)
    return inset

ss = monthly_return(srm)

### processing monthly return


In [3]:
def balance_sheet(inset): #处理原始资产负债表数据，去除金融行业，保留账面价值，总资产等数据
    print("### processing balance sheet data")
    inset['year'] = inset['ACCPER'].apply(lambda x: x.year)
    inset['month'] = inset['ACCPER'].apply(lambda x: x.month)
    
    
    keep = ['STKCD','ACCPER','TYPREP','year','month','at','at_lag','bv','bv_lag','A001000000','A001123000',
            'A001207000','A001212000','A002101000','A002201000','A003000000']
    inset.sort_values(by = ["STKCD","ACCPER"],ascending=True,inplace = True) #ACCPER 公告发表所属的会计周期
    inset['at']=inset['A001000000']
    inset['bv']=inset['A003000000']
    inset['at_lag'] = inset['at'].shift(1)
    inset['bv_lag'] = inset['bv'].shift(1)
    inset.loc[inset.groupby('STKCD')['bv_lag'].head(1).index, 'bv_lag'] = np.nan
    inset.loc[inset.groupby('STKCD')['at_lag'].head(1).index, 'at_lag'] = np.nan  #当期的总资产，总账面价值参考上一期
    A = "A"  #local variable: TYPREP : A
    inset.query("(@enddate>=year>= @begdate)& month==12 & TYPREP == @A",inplace=True)
    inset = inset[keep]
    inset.reset_index(drop = True,inplace=True)
    return inset
bs = balance_sheet(balance)

### processing balance sheet data


In [4]:
def size_month12(inset):    # 使用12月份的市值
    tp = inset.copy()           #使用copy避免srm数据被修改
    keep = ['STKCD', 'TRDMNT', 'year','yearr', 'years','month','mv12']
    tp.query('month==12',inplace=True)
    tp['mv12'] = tp['mv']
    tp['years'] = tp['year']
    tp = tp[keep]
    tp.reset_index(drop = True,inplace=True)
    return tp

ss12 = size_month12(ss)

In [5]:
def bm_ann(inset_balance,inset_mv12):# 计算每年的账面市值比，insets are balance sheet and mv12
    print("### calculating book to market")
    inset_balance = inset_balance[['STKCD','year','bv']]
    inset_mv12 = inset_mv12[['STKCD','year','mv12']]
    ann_bm = pd.merge(inset_balance,inset_mv12,on=['STKCD','year'])
    ann_bm["bm"] = ann_bm['bv']/ann_bm['mv12']
    ann_bm['years'] = ann_bm['year']
    return ann_bm
ss_bm = bm_ann(bs,ss12)

### calculating book to market


In [6]:
def group10(inset,sortvar, var):  # 根据var的十个分位数分10组，做decile投资组合
    print('### grouping')
    quant = [10,20,30,40,50,60,70,80,90]
    grouped = inset.groupby(inset[sortvar])
    perc = pd.DataFrame()
    inset = pd.DataFrame()
    for i in quant:
        perc[str(i)+"%"] = grouped[var].quantile(i/100)
    for year,group in grouped:
        group['group_dum'] = pd.qcut(group[var],10,labels=[1,2,3,4,5,6,7,8,9,10])
        inset = pd.concat([inset,group],join = "outer")
    inset.sort_values(by=[sortvar,'group_dum','STKCD'],inplace = True)
    inset.reset_index(drop = True,inplace = True)
    return inset, perc
gp10, gp10_perc = group10(ss_bm,'years','bm') # 根据book_to_market的十个分位数分10组，做decile投资组合
# gp10_perc 为分位数的集合


### grouping


In [7]:
def stock_ret_month_bm(srm_inset,bm_inset):  #将月收益率和账面市值比合并，使用yearr保证每一期的分组与bm使用的
                                                # 是最近一次年报更新的账面价值和年末时候的市值
    print("### mergeing monthly return data and book to market data")
    srm_inset["merge_key"] = srm_inset['yearr']-1
    bm_inset["merge_key"] = bm_inset['years']
    result = pd.merge(srm_inset,bm_inset,on=['merge_key','STKCD'])
    result = result[["STKCD","TRDMNT",'year_x','month','MRETWD','mv_lag','group_dum','bm','status_decision']]
    strin = "in"
    result.query("status_decision==@strin",inplace = True)
    result = result[["STKCD","TRDMNT",'year_x','month','MRETWD','mv_lag','group_dum','bm']]
    result.sort_values(by=["TRDMNT","STKCD"],inplace=True)
    result.reset_index(drop = True,inplace=True)
    return result
srm_bm = stock_ret_month_bm(ss,gp10)

### mergeing monthly return data and book to market data


In [8]:

def winsorize(inset,sortvar,var,perc,trim):   #使用1%和99%分位数代替离群点的值，trim = 1时候，抛弃离群点
                                            #只能对单变量进行winsorization
    print('### winsorizing')
    tp = inset.copy()
    perc1 = perc
    perc2 = 100-perc
    tp.sort_values(by=sortvar,inplace = True)
    tp.reset_index(drop = True,inplace = True)
    grouped = tp.groupby(sortvar)
    tp = pd.DataFrame()
    if len(sortvar) == 1:
        for i,group in grouped:
            group.sort_values(by=var,inplace=True)
            p1_tp = round(np.percentile(group[var],perc1),2)
            p2_tp = round(np.percentile(group[var],perc2),2)
            if trim == 1:
                group.loc[group[var]<=p1_tp,var] = np.nan
                group.loc[group[var]>=p2_tp,var] = np.nan
            else:
                p1 = np.array(group.loc[group[var]<=p1_tp,var])[-1]
                p2 = np.array(group.loc[group[var]>=p2_tp,var])[0]
                group.loc[group[var]<=p1_tp,var] = p1
                group.loc[group[var]>=p2_tp,var] = p2
            tp = pd.concat([tp,group],join = "outer")
        tp.sort_values(by=list(sortvar)+['STKCD'],inplace = True)
    elif len(sortvar)>1:
        xx = tuple(sortvar)
        for xx,group in grouped:
            group.sort_values(by=var,inplace=True)
            p1_tp = round(np.percentile(group[var],perc1),2)
            p2_tp = round(np.percentile(group[var],perc2),2)
            if trim == 1:
                group.loc[group[var]<=p1_tp,var] = np.nan
                group.loc[group[var]>=p2_tp,var] = np.nan
            else:
                p1 = np.array(group.loc[group[var]<=p1_tp,'mv_lag'])[-1]
                p2 = np.array(group.loc[group[var]>=p2_tp,'mv_lag'])[0]
                group.loc[group['mv_lag']<=p1_tp,var] = p1
                group.loc[group['mv_lag']>=p2_tp,var] = p2
            tp = pd.concat([tp,group],join = "outer")
        tp.sort_values(by=sortvar+['STKCD'],inplace = True)
    tp.reset_index(drop = True,inplace=True)
    return tp[tp[var].notna()]

win1 = winsorize(srm_bm,['TRDMNT'],'mv_lag',1,0)
win2 = winsorize(win1,['group_dum','TRDMNT'],'mv_lag',1,0)

### winsorizing
### winsorizing


In [9]:

def monthly_return_ew(win_ed, sortvar):  #构建等权重投资组合
    grouped = win_ed.groupby(sortvar + ['TRDMNT'])
    xx = tuple(sortvar + ['TRDMNT'])
    tp = []
    for xx, group in grouped:
        row = {}
        row['group_dum'] = xx[0]
        row['TRDMNT'] = xx[1]
        row['freq'] = len(group)
        row['mean'] = np.mean(group['MRETWD'])
        tp.append(row)
    tp = pd.DataFrame(tp)
    tp.sort_values(by=['TRDMNT'] + sortvar, inplace=True)
    tp.reset_index(drop=True, inplace=True)
    return tp


def monthly_return_vw(win_ed, sortvar):  #构建市值加权投资组合
    grouped = win_ed.groupby(sortvar + ['TRDMNT'])
    xx = tuple(sortvar + ['TRDMNT'])
    tp = []
    for xx, group in grouped:
        row = {}
        row['group_dum'] = xx[0]
        row['TRDMNT'] = xx[1]
        row['freq'] = len(group)
        row['mean'] = np.average(group['MRETWD'], weights=group['mv_lag'])
        tp.append(row)
    tp = pd.DataFrame(tp)
    tp.sort_values(by=['TRDMNT'] + sortvar, inplace=True)
    tp.reset_index(drop=True, inplace=True)
    return tp

mon_ret_ew = monthly_return_ew(win2,['group_dum'])
mon_ret_vw = monthly_return_vw(win2,['group_dum'])

In [10]:

def trans(weighted_monthly_return):     #将收益率结果转置
    grouped = weighted_monthly_return.groupby('TRDMNT')
    tp = []
    for month, group in grouped:
        li = list(group['mean'])
        for i in range(len(group)):
            row = {}
            row['TRDMNT'] = month
            row['bm_g10'] = 'col_' + str(i + 1)
            row['MRETWD'] = round(li[i], 9)
            tp.append(row)
        row = {}
        row['TRDMNT'] = month
        row['bm_g10'] = 'high-low'
        row['MRETWD'] = round(li[9] - li[0], 9)
        tp.append(row)
    tp = pd.DataFrame(tp)
    tp.sort_values(by=["bm_g10", "TRDMNT"], inplace=True)
    tp.reset_index(drop=True, inplace=True)
    return tp

mon_ret_ew_trans = trans(mon_ret_ew)
mon_ret_vw_trans = trans(mon_ret_vw)

out_ret = pd.merge(mon_ret_ew_trans,mon_ret_vw_trans,on=['TRDMNT','bm_g10'])
out_ret.rename(columns={'MRETWD_x':'ew', 'MRETWD_y':'vw'}, inplace = True)



grouped = out_ret.groupby('bm_g10')
ou = []
for i, group in grouped:
    row = {}
    row['group_dum'] = i
    row['ew_mean'] = round(np.mean(group['ew']),10)
    row['vw_mean'] = round(np.mean(group['vw']),10)
    ou.append(row)

ou = pd.DataFrame(ou)    #ou整个时间周期的月平均收益率
print(ou)

   group_dum   ew_mean   vw_mean
0      col_1  0.011781  0.006751
1     col_10  0.017709  0.012216
2      col_2  0.013712  0.009827
3      col_3  0.016052  0.010346
4      col_4  0.015283  0.011130
5      col_5  0.017363  0.012912
6      col_6  0.018686  0.012593
7      col_7  0.019454  0.014653
8      col_8  0.019315  0.013261
9      col_9  0.018513  0.012452
10  high-low  0.005928  0.005465


In [11]:
out_ret

,TRDMNT,bm_g10,ew,vw
0,200107.0,col_1,-0.125109,-0.114110
1,200108.0,col_1,-0.021746,-0.030239
2,200109.0,col_1,-0.083686,-0.102126
3,200110.0,col_1,-0.088696,-0.081840
4,200111.0,col_1,0.032551,0.013695
...,...,...,...,...
1909,201508.0,high-low,-0.009986,0.040927
1910,201509.0,high-low,-0.036968,-0.049064
1911,201510.0,high-low,-0.064344,-0.079086
1912,201511.0,high-low,-0.006090,-0.019600


In [12]:
ou

,group_dum,ew_mean,vw_mean
0,col_1,0.011781,0.006751
1,col_10,0.017709,0.012216
2,col_2,0.013712,0.009827
3,col_3,0.016052,0.010346
4,col_4,0.015283,0.011130
5,col_5,0.017363,0.012912
6,col_6,0.018686,0.012593
7,col_7,0.019454,0.014653
8,col_8,0.019315,0.013261
9,col_9,0.018513,0.012452


In [18]:
ff3_data = pd.read_sas('cn_ff3_monthly_95_17.sas7bdat',format='sas7bdat',encoding='iso-8859-1')  
ff3_data['TRDMNT'] = ff3_data['year'] * 100 + ff3_data['month'] 


hml_group = out_ret[out_ret['bm_g10'] == 'high-low']

In [20]:
reg_table = pd.merge(hml_group,ff3_data,on='TRDMNT')

In [58]:
reg_table

,TRDMNT,bm_g10,ew,vw,Date,Rmrf_tmv,Smb_tmv,Hml_tmv,year,month,MonRFRet,EW_rf
0,200107.0,high-low,-0.007832,-0.010282,2001-07-31,-0.1340,-0.0076,-0.0094,2001.0,7.0,0.001650,-0.009482
1,200108.0,high-low,-0.015171,-0.017844,2001-08-31,-0.0378,0.0151,-0.0104,2001.0,8.0,0.001650,-0.016821
2,200109.0,high-low,0.032264,0.060885,2001-09-28,-0.0548,-0.0214,0.0261,2001.0,9.0,0.001650,0.030614
3,200110.0,high-low,0.054867,0.052416,2001-10-31,-0.0506,-0.0157,0.0282,2001.0,10.0,0.001650,0.053217
4,200111.0,high-low,0.022459,0.030285,2001-11-30,0.0371,0.0242,0.0094,2001.0,11.0,0.001650,0.020809
...,...,...,...,...,...,...,...,...,...,...,...,...
169,201508.0,high-low,-0.009986,0.040927,2015-08-31,-0.1468,0.0165,0.0143,2015.0,8.0,0.002592,-0.012578
170,201509.0,high-low,-0.036968,-0.049064,2015-09-30,-0.0535,0.0210,-0.0401,2015.0,9.0,0.002602,-0.039570
171,201510.0,high-low,-0.064344,-0.079086,2015-10-30,0.1413,0.0949,-0.0717,2015.0,10.0,0.002618,-0.066962
172,201511.0,high-low,-0.006090,-0.019600,2015-11-30,0.0394,0.1160,-0.0395,2015.0,11.0,0.002534,-0.008624


In [ ]:
reg_table.to_csv('reg_table.csv')

In [48]:
def GRS_test(factor, resid, alpha):
    N = resid.shape[1]        
    T = resid.shape[0]       
    L = factor.shape[1]      

    if (T-N-L) < 0:
        print('can not conduct GRS test because T-N-L<0')
        return

    factor = np.asmatrix(factor)                   # factor matrix (T, L)
    resid = np.asmatrix(resid)                     # residual matrix (T, N)
    alpha = np.asmatrix(alpha).reshape(N, 1)       # intercept matrix (N, 1)

    mean_return_factor = (factor.mean(axis=0))

    # covariance matrix of residuals
    cov_resid = (resid.T * resid) / (T-L-1)
    # covariance matrix of factors
    cov_factor = ((factor - mean_return_factor).T * (factor - mean_return_factor)) / (T-1)

    mean_return_factor = mean_return_factor.reshape(L, 1)

    # GRS statistic
    f_grs = float((T/N) * ((T-N-L)/(T-L-1)) * ((alpha.T * np.linalg.inv(cov_resid) * alpha) / (1 + mean_return_factor.T * np.linalg.inv(cov_factor) * mean_return_factor)))

    # p-value
    p_grs = 1 - st.f.cdf(f_grs, N, (T-N-L))

    return f_grs, p_grs

In [30]:
reg_table['EW_rf'] = reg_table['ew'] - reg_table['MonRFRet']
mod = smf.ols(formula=' EW_rf ~ 1 + Rmrf_tmv', data=reg_table)
reg_res = mod.fit()

In [32]:
reg_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  EW_rf   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     4.811
Date:                Sun, 28 Nov 2021   Prob (F-statistic):             0.0296
Time:                        22:45:21   Log-Likelihood:                 330.85
No. Observations:                 174   AIC:                            -657.7
Df Residuals:                     172   BIC:                            -651.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0028      0.003      1.009      0.315      -0.003       0.008
Rmrf_tmv       0.0687      0.031      2.193      0.030       0.007       0.130
==============================================================================
Omnibus:                        1.263   Durbin-Watson:                   2.065
Prob(Omnibus):                  0.532   Jarque-Bera (JB):                0.885
Skew:                           0.116   Prob(JB):                        0.642
Kurtosis:                       3.262   Cond. No.                         11.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [49]:
resid_matrix = pd.DataFrame(reg_res.resid).values
factors_matrix = reg_table[['Rmrf_tmv']].values
alpha_matrix = pd.DataFrame([reg_res.params['Intercept']]).values

In [55]:
grs_stat, p_values = GRS_test(factors_matrix, resid_matrix, alpha_matrix)

In [56]:
grs_stat

1.0174384078336838

In [59]:
p_values

0.31454428750536734